In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,auc,average_precision_score,balanced_accuracy_score,brier_score_loss,classification_report,cohen_kappa_score,confusion_matrix,dcg_score,det_curve,f1_score,fbeta_score,hamming_loss,hinge_loss,jaccard_score,log_loss,matthews_corrcoef,multilabel_confusion_matrix,ndcg_score,precision_recall_curve,precision_recall_fscore_support,precision_score,recall_score,roc_auc_score,roc_curve,top_k_accuracy_score,zero_one_loss
from sklearn.model_selection import train_test_split
from scipy.misc import electrocardiogram
import pywt
import scipy.signal as signal
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [2]:
hemiplegiaFiles = os.listdir('../Data/Hemiplegia')
normalFiles = os.listdir('../Data/Normal')
hemiplegia = pd.DataFrame()
normal = pd.DataFrame()
for i in hemiplegiaFiles:
    df = pd.read_csv('../Data/Hemiplegia/' + i)
    df.columns = ['index','Ax','Ay','Az','Gx','Gy','Gz']
    hemiplegia = pd.concat([hemiplegia,df],axis=0)
for i in normalFiles:
    df = pd.read_csv('../Data/Normal/' + i)
    df.columns = ['index','Ax','Ay','Az','Gx','Gy','Gz']
    normal = pd.concat([normal,df],axis=0)
hemiplegia = hemiplegia.drop(columns='index')
hemiplegia['Diagnosis'] = 'Hemiplegia'
normal = normal.drop(columns='index')
normal['Diagnosis'] = 'Normal'

In [3]:
cols = ['Ax','Ay','Az','Gx','Gy','Gz']
dfDWT = pd.DataFrame()
dfButterworth = pd.DataFrame()
dfGeneralDWT = pd.DataFrame()
data = normal
for col in cols:
    wavelet_type='db6'
    DWTcoeffs = pywt.wavedec(data[col],wavelet_type,mode='symmetric', level=9, axis=-1)
    DWTcoeffs[-1] = np.zeros_like(DWTcoeffs[-1])
    DWTcoeffs[-2] = np.zeros_like(DWTcoeffs[-2])
    DWTcoeffs[-3] = np.zeros_like(DWTcoeffs[-3])
    DWTcoeffs[-4] = np.zeros_like(DWTcoeffs[-4])
    DWTcoeffs[-5] = np.zeros_like(DWTcoeffs[-5])
    DWTcoeffs[-6] = np.zeros_like(DWTcoeffs[-6])
    DWTcoeffs[-7] = np.zeros_like(DWTcoeffs[-7])
    DWTcoeffs[-8] = np.zeros_like(DWTcoeffs[-8])
    DWTcoeffs[-9] = np.zeros_like(DWTcoeffs[-9])
    filtered_data_dwt=pywt.waverec(DWTcoeffs,wavelet_type,mode='symmetric',axis=-1) 
    fc = 0.9  # Cut-off frequency of the butterworth filter
    w = fc / (100 / 2) # Normalize the frequency
    b, a = signal.butter(5, w, 'low')
    filtered_data_butterworth = signal.filtfilt(b, a, data[col])
    tempDWT = pd.DataFrame(filtered_data_dwt)
    tempDWT.columns = [col]
    dfDWT = pd.concat([dfDWT,tempDWT],axis=1)
    tempButterworth = pd.DataFrame(filtered_data_butterworth)
    tempButterworth.columns = [col]
    dfButterworth = pd.concat([dfButterworth,tempButterworth],axis=1)
    genDWTcoeffs = pywt.wavedec(data[col],wavelet_type,mode='symmetric', level=9, axis=-1)
    filtered_data_gen_dwt=pywt.waverec(genDWTcoeffs,wavelet_type,mode='symmetric',axis=-1)
    tempgenDWT = pd.DataFrame(filtered_data_gen_dwt)
    tempgenDWT.columns = [col]
    dfGeneralDWT = pd.concat([dfGeneralDWT,tempgenDWT],axis=1)
    
dfDWT[['Diagnosis']] = 'Normal'
dfButterworth[['Diagnosis']] = 'Normal'
dfDWTNormal = dfDWT
dfButterworthNormal = dfButterworth
dfGeneralDWT[['Diagnosis']] = 'Normal'
generalDWT = dfGeneralDWT



cols = ['Ax','Ay','Az','Gx','Gy','Gz']
dfDWT = pd.DataFrame()
dfButterworth = pd.DataFrame()
dfGeneralDWT = pd.DataFrame()
data = hemiplegia
for col in cols:
    wavelet_type='db6'
    DWTcoeffs = pywt.wavedec(data[col],wavelet_type,mode='symmetric', level=9, axis=-1)
    DWTcoeffs[-1] = np.zeros_like(DWTcoeffs[-1])
    DWTcoeffs[-2] = np.zeros_like(DWTcoeffs[-2])
    DWTcoeffs[-3] = np.zeros_like(DWTcoeffs[-3])
    DWTcoeffs[-4] = np.zeros_like(DWTcoeffs[-4])
    DWTcoeffs[-5] = np.zeros_like(DWTcoeffs[-5])
    DWTcoeffs[-6] = np.zeros_like(DWTcoeffs[-6])
    DWTcoeffs[-7] = np.zeros_like(DWTcoeffs[-7])
    DWTcoeffs[-8] = np.zeros_like(DWTcoeffs[-8])
    DWTcoeffs[-9] = np.zeros_like(DWTcoeffs[-9])
    filtered_data_dwt=pywt.waverec(DWTcoeffs,wavelet_type,mode='symmetric',axis=-1) 
    fc = 0.9  # Cut-off frequency of the butterworth filter
    w = fc / (100 / 2) # Normalize the frequency
    b, a = signal.butter(5, w, 'low')
    filtered_data_butterworth = signal.filtfilt(b, a, data[col])
    tempDWT = pd.DataFrame(filtered_data_dwt)
    tempDWT.columns = [col]
    dfDWT = pd.concat([dfDWT,tempDWT],axis=1)
    tempButterworth = pd.DataFrame(filtered_data_butterworth)
    tempButterworth.columns = [col]
    dfButterworth = pd.concat([dfButterworth,tempButterworth],axis=1)
    genDWTcoeffs = pywt.wavedec(data[col],wavelet_type,mode='symmetric', level=9, axis=-1)
    filtered_data_gen_dwt=pywt.waverec(genDWTcoeffs,wavelet_type,mode='symmetric',axis=-1)
    tempgenDWT = pd.DataFrame(filtered_data_gen_dwt)
    tempgenDWT.columns = [col]
    dfGeneralDWT = pd.concat([dfGeneralDWT,tempgenDWT],axis=1)
dfDWT[['Diagnosis']] = 'Hemiplegia'
dfButterworth[['Diagnosis']] = 'Hemiplegia'
dfDWThemiplegia = dfDWT
dfButterworthHemiplegia = dfButterworth
dfGeneralDWT[['Diagnosis']] = 'Hemiplegia'
generalDWTHemi = dfGeneralDWT

combinedDWT = pd.concat([dfDWTNormal,dfDWThemiplegia],axis=0).sample(frac=1).reset_index(drop=True)
combinedButterworth = pd.concat([dfButterworthNormal,dfButterworthHemiplegia],axis=0).sample(frac=1).reset_index(drop=True)
combinedDWTgen = pd.concat([generalDWT,generalDWTHemi],axis=0).sample(frac=1).reset_index(drop=True)

In [61]:
print('For DWT:\n\n')
print('GaussianNB:')
df = combinedDWT
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = GaussianNB()
model.fit(data_train, labels_train)

pred = model.predict(data_test)
labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

print('Logistic Regression')
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = LogisticRegression(random_state=0)
model.fit(data_train, labels_train)

labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = model.predict(data_test)
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

print('Random Forest')
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = RandomForestClassifier(random_state=0)
model.fit(data_train, labels_train)

labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = model.predict(data_test)
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

print('Decision Tree')
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = DecisionTreeClassifier(random_state=0)
model.fit(data_train, labels_train)

labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = model.predict(data_test)
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

For DWT:


GaussianNB:
0.8865336228587185
0.851460898634717
0.9059982486865149
0.9475684586500596
0.8241675045863539
0.1134663771412815
Logistic Regression


/home/subhrangshu/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/subhrangshu/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/subhrangshu/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/

0.8696791093647676
0.8454622180414302
0.855324849296718
0.9950041250343753
0.7423248561238589
0.13032089063523247
Random Forest
1.0
1.0
1.0
1.0
1.0
0.0
Decision Tree
0.9996897942301727
0.9996525053522496
0.9996801462188714
0.999908592321755
0.9994636190264319
0.0003102057698273188


In [63]:
print('For Butterworth:\n\n')
print('GaussianNB:')
df = combinedButterworth
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = GaussianNB()
model.fit(data_train, labels_train)

pred = model.predict(data_test)
labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

print('Logistic Regression')
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = LogisticRegression(random_state=0)
model.fit(data_train, labels_train)

labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = model.predict(data_test)
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

print('Random Forest')
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = RandomForestClassifier(random_state=0)
model.fit(data_train, labels_train)

labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = model.predict(data_test)
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

print('Decision Tree')
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = DecisionTreeClassifier(random_state=0)
model.fit(data_train, labels_train)

labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = model.predict(data_test)
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

For Butterworth:


GaussianNB:
0.8270430496673905
0.7625498474157248
0.8638892482856774
0.9149499840131549
0.7358457785234314
0.17295695033260952
Logistic Regression


/home/subhrangshu/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/subhrangshu/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8394168131527247
0.7861916896937206
0.8364355970061523
0.9777828676133761
0.6983443086500682
0.16058318684727535
Random Forest


/tmp/ipykernel_6008/2496967286.py:80: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(data_train, labels_train)


0.9954847826836246
0.9953875771923091
0.9946092865232163
0.9994537509104151
0.9912811692806157
0.004515217316375418
Decision Tree
0.9833522903526005
0.97851632756517
0.987178321360371
0.9908273627527039
0.9755545264467744
0.016647709647399443


In [4]:
print('For General DWT:\n\n')
print('GaussianNB:')
df = combinedDWTgen
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = GaussianNB()
model.fit(data_train, labels_train)

pred = model.predict(data_test)
labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

print('Logistic Regression')
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = LogisticRegression(random_state=0)
model.fit(data_train, labels_train)

labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = model.predict(data_test)
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

print('Random Forest')
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = RandomForestClassifier(random_state=0)
model.fit(data_train, labels_train)

labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = model.predict(data_test)
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

print('Decision Tree')
data = df[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = df[['Diagnosis']]
data_train, data_test, labels_train, labels_test = train_test_split(data, labels)
model = DecisionTreeClassifier(random_state=0)
model.fit(data_train, labels_train)

labels_test = pd.Categorical(labels_test['Diagnosis']).codes
pred = model.predict(data_test)
pred = pd.Categorical(pred).codes

print(accuracy_score(labels_test, pred))
print(fbeta_score(labels_test, pred, average='macro', beta=0.5))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))
print(roc_auc_score(labels_test, pred))
print(brier_score_loss(labels_test, pred))

For General DWT:


GaussianNB:
0.8296280977492848
0.7712221267695964
0.8795528308690949
0.8957443878253684
0.7630865784216745
0.1703719022507152
Logistic Regression


/home/subhrangshu/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/subhrangshu/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7522145245234895
0.5452784874614603
0.7676557538596299
0.9623444363103953
0.5364328021987914
0.24778547547651053
Random Forest


/tmp/ipykernel_25907/2196338785.py:80: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(data_train, labels_train)


0.9613621480026195
0.949634845142491
0.9717183716361311
0.9769407441433164
0.9457394594663842
0.038637851997380485
Decision Tree
0.9293420190948884
0.9056859137614273
0.9535719220671549
0.9520809381467004
0.9066953947750563
0.0706579809051115
